In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# world = client.get_world()
world = client.load_world("Town04")
spectator = world.get_spectator()

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

map = world.get_map()
map

In [ ]:
target_vehicle = spawn_vehicle()

time.sleep(1)

#target_transform = target_vehicle.get_transform()
#target_transform.location += carla.Location(x=100)
#target_vehicle.set_transform(target_transform)

ego_vehicle = spawn_vehicle()
ego_vehicle_transform = ego_vehicle.get_transform()
ego_vehicle_transform.location = carla.Location(x=283, y=-206)
ego_vehicle.set_transform(ego_vehicle_transform)

camera = spawn_camera(attach_to=ego_vehicle)

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

camera.listen(lambda image: camera_callback(image))

# target_vehicle.set_autopilot(True)

ego_vehicle.apply_control(carla.VehicleControl(throttle = 1))

# target_velocity = carla.Vector3D(x=20) # Set the target velocity to 20 m/s
# ego_vehicle.set_target_velocity(target_velocity)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    target_vehicle.destroy()
    ego_vehicle.destroy()

In [ ]:
target_vehicle = spawn_vehicle()
time.sleep(1)
print(target_vehicle.get_transform().location)
print(world.get_map().get_spawn_points()[0])

target_vehicle_transform = target_vehicle.get_transform()
target_vehicle_transform.location = carla.Location(x=280, y=-207.5)
target_vehicle_transform.rotation = carla.Rotation(yaw=180)
target_vehicle.set_transform(target_vehicle_transform)

In [ ]:
cv2.destroyAllWindows()
camera.destroy()
target_vehicle.destroy()
ego_vehicle.destroy()

In [ ]:
for actor in world.get_actors().filter("vehicle.*"):
    actor.destroy()